In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda --version

✨🍰✨ Everything looks OK!
conda 23.11.0


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [13]:
!conda install pytorch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 pytorch-cuda=12.1 torchdata -c pytorch -c nvidia

Channels:
 - pytorch
 - nvidia
 - conda-forge
 - dglteam/label/th21_cu121
Platform: linux-64
Solving environment: \ | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pytorch-cuda=12.1
    - pytorch==2.1.0
    - torchaudio==2.1.0
    - torchdata
    - torchvision==0.16.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    torchdata-0.7.1            |            py310         2.4 MB  pytorch
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

  torchdata          pytorch/linux-64::torchdata-0.7.1-py310 



              

In [10]:
!conda install -c dglteam/label/th21_cu121 dgl

Channels:
 - dglteam/label/th21_cu121
 - conda-forge
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: / - \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [4]:
import torch;
print(torch.__version__)
print(torch.cuda.is_available()) # should be True
t = torch.rand(10, 10).cuda()
print(t.device) # should be CUDA

2.1.0
True
cuda:0


In [5]:
import os
os.chdir('/content/drive/MyDrive/antifraud/')

In [6]:
import os
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
from config import Config
from feature_engineering.data_engineering import data_engineer_benchmark, span_data_2d, span_data_3d
import logging
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import pickle
from scipy.io import loadmat
import yaml

logger = logging.getLogger(__name__)

In [14]:
import dgl

In [15]:
def base_load_data(args: dict):
    # load S-FFSD dataset for base models
    data_path = "data/S-FFSD.csv"
    feat_df = pd.read_csv(data_path)
    train_size = 1 - args['test_size']
    method = args['method']
    # for ICONIP16 & AAAI20
    if args['method'] == 'stan':
        if os.path.exists("data/tel_3d.npy"):
            return
        features, labels = span_data_3d(feat_df)
    else:
        if os.path.exists("data/tel_2d.npy"):
            return
        features, labels = span_data_2d(feat_df)
    num_trans = len(feat_df)
    trf, tef, trl, tel = train_test_split(
        features, labels, train_size=train_size, stratify=labels, shuffle=True)
    trf_file, tef_file, trl_file, tel_file = args['trainfeature'], args[
        'testfeature'], args['trainlabel'], args['testlabel']

    np.save(trf_file, trf)
    np.save(tef_file, tef)
    np.save(trl_file, trl)
    np.save(tel_file, tel)
    return

In [16]:
def main(args):
    if args['method'] == 'mcnn':
        from methods.mcnn.mcnn_main import mcnn_main
        base_load_data(args)
        mcnn_main(
            args['trainfeature'],
            args['trainlabel'],
            args['testfeature'],
            args['testlabel'],
            epochs=args['epochs'],
            batch_size=args['batch_size'],
            lr=args['lr'],
            device=args['device']
        )
    elif args['method'] == 'stan_2d':
        from methods.stan.stan_2d_main import stan_main
        base_load_data(args)
        stan_main(
            args['trainfeature'],
            args['trainlabel'],
            args['testfeature'],
            args['testlabel'],
            mode='2d',
            epochs=args['epochs'],
            batch_size=args['batch_size'],
            attention_hidden_dim=args['attention_hidden_dim'],
            lr=args['lr'],
            device=args['device']
        )
    elif args['method'] == 'stan':
        from methods.stan.stan_main import stan_main
        base_load_data(args)
        stan_main(
            args['trainfeature'],
            args['trainlabel'],
            args['testfeature'],
            args['testlabel'],
            mode='3d',
            epochs=args['epochs'],
            batch_size=args['batch_size'],
            attention_hidden_dim=args['attention_hidden_dim'],
            lr=args['lr'],
            device=args['device']
        )

    elif args['method'] == 'stagn':
        from methods.stagn.stagn_main import stagn_main, load_stagn_data
        features, labels, g = load_stagn_data(args)
        stagn_main(
            features,
            labels,
            args['test_size'],
            g,
            mode='2d',
            epochs=args['epochs'],
            attention_hidden_dim=args['attention_hidden_dim'],
            lr=args['lr'],
            device=args['device']
        )
    elif args['method'] == 'gtan':
        from methods.gtan.gtan_main import gtan_main, load_gtan_data
        feat_data, labels, train_idx, test_idx, g, cat_features = load_gtan_data(
            args['dataset'], args['test_size'])
        gtan_main(
            feat_data, g, train_idx, test_idx, labels, args, cat_features)
    elif args['method'] == 'rgtan':
        from methods.rgtan.rgtan_main import rgtan_main, loda_rgtan_data
        feat_data, labels, train_idx, test_idx, g, cat_features, neigh_features = loda_rgtan_data(
            args['dataset'], args['test_size'])
        rgtan_main(feat_data, g, train_idx, test_idx, labels, args,
                   cat_features, neigh_features, nei_att_head=args['nei_att_heads'][args['dataset']])
    else:
        raise NotImplementedError("Unsupported method. ")

In [17]:
def parse_args():
    parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter,
                            conflict_handler='resolve')
    parser.add_argument("--method", default=str)  # specify which method to use
    method = vars(parser.parse_args())['method']  # dict

    # if method in ['']:
    #     yaml_file = "config/base_cfg.yaml"
    if method in ['mcnn']:
        yaml_file = "config/mcnn_cfg.yaml"
    elif method in ['stan']:
        yaml_file = "config/stan_cfg.yaml"
    elif method in ['stan_2d']:
        yaml_file = "config/stan_2d_cfg.yaml"
    elif method in ['stagn']:
        yaml_file = "config/stagn_cfg.yaml"
    elif method in ['gtan']:
        yaml_file = "config/gtan_cfg.yaml"
    elif method in ['rgtan']:
        yaml_file = "config/rgtan_cfg.yaml"
    else:
        raise NotImplementedError("Unsupported method.")

    # config = Config().get_config()
    with open(yaml_file) as file:
        args = yaml.safe_load(file)
    args['method'] = method

    # WIP, add prefix of path; device to use "cuda:0"

    return args

In [18]:
sys.argv = ['main.py', '--method', 'mcnn']
main(parse_args())

100%|██████████| 29643/29643 [24:12<00:00, 20.40it/s]
/usr/local/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Epoch: 0, loss: 1.2741, auc: 0.6433, F1: 0.5395, AP: 0.2393
Epoch: 1, loss: 0.9441, auc: 0.6695, F1: 0.5496, AP: 0.2540
Epoch: 2, loss: 0.8201, auc: 0.6590, F1: 0.5088, AP: 0.2443
Epoch: 3, loss: 0.9220, auc: 0.6463, F1: 0.4760, AP: 0.2360
Epoch: 4, loss: 1.5775, auc: 0.6599, F1: 0.5125, AP: 0.2451
Epoch: 5, loss: 1.1834, auc: 0.6925, F1: 0.5795, AP: 0.2714
Epoch: 6, loss: 0.9784, auc: 0.6931, F1: 0.5533, AP: 0.2671
Epoch: 7, loss: 1.7334, auc: 0.6918, F1: 0.5892, AP: 0.2731
Epoch: 8, loss: 1.0412, auc: 0.7043, F1: 0.6448, AP: 0.2989
Epoch: 9, loss: 0.8395, auc: 0.6903, F1: 0.6154, AP: 0.2795
Epoch: 10, loss: 0.6622, auc: 0.6851, F1: 0.6011, AP: 0.2724
Epoch: 11, loss: 0.5950, auc: 0.7067, F1: 0.6732, AP: 0.3178
Epoch: 12, loss: 0.6719, auc: 0.7026, F1: 0.6700, AP: 0.3137
Epoch: 13, loss: 0.7234, auc: 0.7044, F1: 0.6423, AP: 0.2978
Epoch: 14, loss: 0.7256, auc: 0.7000, F1: 0.6525, AP: 0.3010
Epoch: 15, loss: 0.5922, auc: 0.7003, F1: 0.6643, AP: 0.3086
Epoch: 16, loss: 0.6801, auc: 0.70